---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [6]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)


In [7]:
def convert_month(string):
    month_map = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
         'may':5,
         'jun':6,
         'jul':7,
         'aug':8,
         'sep':9,
         'oct':10,
         'nov':11,
         'dec':12
        }
    s = string.strip()[:3].lower()
    converted = month_map[s]
    return converted


In [8]:
def date_sorter():
    
    format_1 = df.str.extract(r'(?P<Month>\d{1,2})[/-](?P<Day>\d{1,2})[/-](?P<Year>\d{2,4})').dropna()
    
    format_2 = df.str.extract(r'(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[ ,.-]+(?P<Day>\d{2,4})[ .,-]+(?P<Year>\d{2,4})').dropna()
    for index, row in format_2.iterrows():
        row['Month'] = convert_month(row['Month'])
        
    format_3 = df.str.extract(r'(?P<Day>\d{1,2})[ ,.-]?(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[ .,-](?P<Year>\d{2,4})').dropna()
    for index, row in format_3.iterrows():
        row['Month'] = convert_month(row['Month'])
    
    format_4 = df.str.extract(r'(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[ ,.-]+(?P<Day>\d{2,4})[st th rd][ .,-]+(?P<Year>\d{2,4})').dropna()
    
    format_5 = df.str.extract(r'\D(?P<Day> )(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*),?[ ](?P<Year>\d{4})').dropna()
    for index, row in format_5.iterrows():
        row['Month'] = convert_month(row['Month'])
        
    format_51 = df.str.extract(r'^[\D\. ]?(?P<Day>)(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,.\s]*(?P<Year>\d{4})').dropna()
    for index, row in format_51.iterrows():
        row['Month'] = convert_month(row['Month'])
    
    format_52 = df.str.extract(r'[^]*[\D\. ](?P<Day>)(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,. ]*(?P<Year>\d{4})\D').dropna()
    for index, row in format_52.iterrows():
        row['Month'] = convert_month(row['Month'])
        
    format_6 = df.str.extract(r'(?:^| |~|[a-z])+-?(?P<Day>)(?P<Month>\d{1,2})[/-](?P<Year>\d{4})').dropna()
    
    dfs = [format_1, format_2, format_3, format_4, format_5, format_51, format_52, format_6]
    df_final = pd.concat(dfs)
    df_final = df_final.replace(r'^\s*$', 1, regex=True)

    
    for index, row in df_final.iterrows():
        if len(row['Year'])<3 :
          row['Year'] = '19' + row['Year']
        if len(str(row['Day'])) == 1 :
          row['Day'] = '0' + str(row['Day'])
        if len(str(row['Month'])) == 1 :
          row['Month'] = '0' + str(row['Month'])
    
    df_final = df_final.applymap(str)
    
    df_final["combine"] =  df_final["Year"] + df_final["Month"] + df_final["Day"]

    df_final['combine'] = df_final['combine'].astype(int)
    
    df_final['combine'] = pd.to_datetime(df_final['combine'], format='%Y%m%d')
    
    ans = pd.to_datetime(df_final['combine'], format='%Y%m%d')
    ans = pd.DataFrame(ans)
    ans = ans.sort_values(ascending=True, by = 'combine')
    ans.reset_index(inplace=True)
    
    return ans['index']
    


In [9]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
442    231
443    141
444    186
445    161
446    413
Name: index, Length: 447, dtype: int64

TypeError: unhashable type: 'list'